# CarStore3000 Data Science Project

### Objective: To provide an algorithm capable of predicting which used cars will sell the fastest.


## Data loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('VO_annonces_2015-ALL_CR.csv')
display(df.head())
display(df.info())


<ipython-input-15-9315c690edef>:6: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('VO_annonces_2015-ALL_CR.csv')


## Data exploration



In [ ]:
print(df.dtypes)
print(df.describe(include='all'))

#unique values
for col in df.columns:
    print(f"Unique values in '{col}': {df[col].nunique()}")

# inspecting relevant columns
print(df[['date_created', 'date_last_seen', 'price_eur']].describe(include='all'))


In [ ]:

# Convert numeric columns to numeric types
numeric_cols = ['mileage', 'manufacture_year', 'engine_displacement', 'engine_power', 'door_count', 'seat_count', 'price_eur']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Handle price_eur outliers (replace unrealistic values with NaN)
df.loc[df['price_eur'] < 100, 'price_eur'] = np.nan

# Convert date columns to datetime objects
df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')
df['date_last_seen'] = pd.to_datetime(df['date_last_seen'], errors='coerce')

# Display basic info again
display(df.info())
display(df.describe())

# Number of ads (rows)
num_ads = len(df)
print(f"\nNumber of ads scanned: {num_ads}")

## Data cleaning
Handling missing values, converting data types, and filtering to year 2016.


In [ ]:
# Removing Irrelevant Columns to narrow data set for efficiency
irrelevant_cols = ['color_slug', 'stk_year', 'door_count', 'seat_count','fuel_type']
df.drop(columns=irrelevant_cols, inplace=True, errors='ignore')

# Changing to correct typer
for col in ['mileage', 'manufacture_year', 'engine_displacement', 'price_eur']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')
df['date_last_seen'] = pd.to_datetime(df['date_last_seen'], errors='coerce')

df = df.drop_duplicates()

# Removing invalid rows
df = df.dropna(subset=["price_eur", "date_created", "date_last_seen","maker"])



## Data wrangling

Creating new features, such as ad duration, car age, and a flag for "forgotten" ads.


In [ ]:
# Ad Duration
df["days_online"] = (df['date_last_seen'] - df['date_created']).dt.days
df = df[df["days_online"] > 0]

# Calculating Car Age
df['car_age'] = df['date_created'].dt.year - df['manufacture_year']

# histogram of days online
plt.figure(figsize=(10,5))
sns.histplot(df["days_online"], bins=50, kde=True)
plt.xlabel("Days Online")
plt.ylabel("Frequency")
plt.title("Distribution of Days Online (Used Car Listings in 2016)")
plt.show()

threshold_forgotten = df["days_online"].quantile(0.99) #175 days

# Identify "Forgotten" Ads with threshold estimated around 67.5 days & 145 days - points of major drop off - indicative of removed ads & hence forgotten ads)
## Assigns boolean (T/F) based on whether it is forgotten or not
df['forgotten_ad'] = df["days_online"] > 145
df['forgotten_ad'] = (df["days_online"] > 67.5) & (df["days_online"] < 80)

# filtering for 2016
df['year_created'] = df['date_created'].dt.year
df_2016 = df[df['year_created'] == 2016].copy()

display(df_2016.head())

## Data analysis


In [ ]:
# Q1: Total Ads Scanned
total_ads = len(df)
print("="*50)
print(f"Total Ads Correctly Scanned: {total_ads}")

# Q2: Total Used Cars Sold via Classified Ads in 2016
df_2016 = df[df["date_last_seen"].dt.year == 2016]
total_sold_2016 = len(df_2016)
print("="*50)
print(f"Total Used Cars Sold via Classified Ads in 2016: {total_sold_2016}")

# Q3: Market Size (Total Value of Used Cars Sold in 2016)
market_size_2016 = df_2016["price_eur"].sum()
print("="*50)
print(f"Market Size in 2016: €{market_size_2016:,.2f}")

# Q4: Sum of Value Purchased for Each Brand
brand_value = df.groupby("maker")["price_eur"].sum()
print("="*50)
print("Sum of Value Purchased for Each Brand:")
print(brand_value.to_string())

# Q5: Market Share of Each Brand
market_share = (brand_value / df["price_eur"].sum()) * 100
print("="*50)
print("Market Share of Each Brand (%):")
print(market_share.to_string())


# Q6: Average Days Online in 2016
avg_days_online_2016 = df_2016["days_online"].mean()
print("="*50)
print(f"Average Days Online in 2016: {avg_days_online_2016:.2f} days")

# Q7: Average age of used cars sold in 2016
df_sold_2016 = df_2016[df_2016["date_last_seen"].dt.year < 2017]
avg_car_age_2016 = df_sold_2016["car_age"].mean()
print("="*50)
print(f"Average Age of Used Cars Sold in 2016: {avg_car_age_2016:.2f} years")

# Q8: Estimated number of forgotten ads
num_forgotten_ads = df[df['forgotten_ad']].shape[0]
print("="*50)
print(f"Estimated Forgotten Ads: {num_forgotten_ads}")

# Q9: Average Time to Sell (excluding forgotten ads)
df_filtered = df_2016[~df_2016['forgotten_ad']]
avg_days_online_filtered = df_filtered["days_online"].mean()
print("="*50)
print(f"Average Days to Sell (Excluding Forgotten Ads): {avg_days_online_filtered:.2f} days")

# Q10: Five Fastest Selling Models in 2016
fastest_models = df_filtered.groupby("model")["days_online"].mean().nsmallest(5)
print("="*50)
print("Five Fastest Selling Models in 2016 (Excluding Forgotten Ads):")
print(fastest_models.to_string())


## Data visualization

In [ ]:
#Market Share by Brand
plt.figure(figsize=(12, 6))
market_share .sort_values(ascending=False).plot(kind='bar')
plt.title('Market Share by Brand (2016)')
plt.xlabel('Brand')
plt.ylabel('Market Share')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


# Distribution of Ad Duration
plt.figure(figsize=(10, 5))
plt.hist(df_2016['days_online'], bins=50, color='skyblue', edgecolor='black')
plt.axvline(df_2016['days_online'].mean(), color='red', linestyle='dashed', linewidth=1, label='Average Ad Duration')
plt.title('Distribution of Ad Durations (2016)')
plt.xlabel('Ad Duration (Days)')
plt.ylabel('Number of Ads')
plt.legend()
plt.tight_layout()
plt.show()


# Distribution of Time to Sell (excluding "forgotten" ads)
plt.figure(figsize=(10, 5))
plt.hist(df_2016.loc[~df_2016['forgotten_ad'], 'days_online'], bins=30, color='lightgreen', edgecolor='black')
plt.axvline(df_2016.loc[~df_2016['forgotten_ad'], 'days_online'].mean(), color='red', linestyle='dashed', linewidth=1, label='Average Time to Sell')
plt.title('Distribution of Time to Sell (2016, Excluding Forgotten Ads)')
plt.xlabel('Time to Sell (Days)')
plt.ylabel('Number of Ads')
plt.legend()
plt.tight_layout()
plt.show()


## Data Analysis Summary:

### 1. Data Analysis Key Findings

* **Market Dominance:** BMW and Renault held the largest market share (~46.6% and ~47%) in 2016.
* **Significant "Forgotten" Ads:** A substantial number of ads were classified as "forgotten," indicating a potential issue with ad management or prolonged sales cycles.
* **High Market Value:** The total market value of used cars sold in 2016 was exceptionally high, suggesting a large and active used car market.


### 2. Insights or Next Steps

* **Inventory Strategy:** CarStore3000 should prioritize stocking the fastest-selling models (418i, Siena, 730i, Tacoma, and e300-td) identified in the analysis, but also consider expanding the list beyond the top 5 to ensure broader market coverage.  They should explore the reasons behind the "forgotten" ads and see if improvements to their own sales process can reduce the number of ads with prolonged sales cycles.

# Machine Learning Models for Prediction


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

features = ["maker", "model", "mileage", "manufacture_year", "engine_displacement",
            "engine_power", "transmission", "price_eur"]

df_model = df_2016[features + ["days_online"]].dropna()

df_model = df_model.sample(n=10_000, random_state=42)

df_model = pd.get_dummies(df_model, columns=["maker", "model", "transmission"])

X = df_model.drop("days_online", axis=1)
y = df_model["days_online"]

#Spliting data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Evaluation
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mae, r2

models = {"Random Forest": model, "Linear Regression": linear_model}
print("="*50)
print("Model Evaluation Results")
print("="*50)

for name, model in models.items():
    mae, r2 = evaluate_model(model, X_test, y_test)
    print(f"{name}:")
    print(f"   Mean Absolute Error (MAE): {mae:.2f} days")
    print(f"   R² Score: {r2:.4f}")
    print("-"*50)



## Model Evaluation Results

Random Forest:
   Mean Absolute Error (MAE): 32.55 days
   R² Score: 0.1991

Linear Regression:
   Mean Absolute Error (MAE): 38.25 days
   R² Score: -0.1642

Therefore Random Forest will be a better model and is a better fit for the data given it's positive R^2 score and lower Mean Absolute Error.

## Attractiveness Score Prediction

In [ ]:
max_days_online = df_2016["days_online"].max()
min_days_online = df_2016["days_online"].min()
avg_days_online = df_2016["days_online"].mean()

# attractiveness Score Function
def compute_attractiveness(days):
    return np.tanh((avg_days_online - days) / (max_days_online - min_days_online))

rf_attractiveness = np.array([compute_attractiveness(days) for days in y_pred_rf])
lr_attractiveness = np.array([compute_attractiveness(days) for days in y_pred_lr])

#  sample results
print("="*50)
print("Random Forest - Sample Predictions and Attractiveness Scores")
for i in range(5):
    print(f"Predicted Days: {y_pred_rf[i]:.2f} -> Attractiveness Score: {rf_attractiveness[i]:.4f}")

print("="*50)
print("Linear Regression - Sample Predictions and Attractiveness Scores")
for i in range(5):
    print(f"Predicted Days: {y_pred_lr[i]:.2f} -> Attractiveness Score: {lr_attractiveness[i]:.4f}")

